# web scraping RIP.ie with beautifulsoup

In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n

    user_agent = 'Chrome/107.0.5304.110'
    headers = {'User-Agent': user_agent}
    req = requests.get(url,headers=headers)   
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.title == None:
        title = ''
    else:        
        title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [5]:
get_dn_page(524700)

('', '', '', '', '')

In [3]:
#df = pd.read_csv('rip_dn_scrape.csv')
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

484851

## iterate over a range of ids and get info

In [6]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
print (len(df))
ids = list(df.id)

results={}
for n in range(521000,524700):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)
    if name == '':
        continue
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

523066
524061 2023-01-25 00:00:00 Lynette  Morris (née Betts) Chestnut Grove, Castlebar,  Mayo Mayo
524062 2023-01-26 00:00:00 Madeline  Bevans Drumcondra,  Dublin Dublin
524063 2023-01-25 00:00:00 Charlie  Keaney Derrin, Ballinagh,  Cavan Cavan
524064 2023-01-25 00:00:00 Maureen  Healy (née Keady) Roscam,  Galway / Headford, Galway Galway
524065 2023-01-24 00:00:00 Brian  Douglas Firhouse,  Dublin / Booterstown, Dublin Dublin
524066 2023-01-22 00:00:00 Geraldine  O'REILLY (née Crone) Rathfarnham,  Dublin Dublin
524067 2023-01-25 00:00:00 SR THERESA  BURKE St Marie's of the Isle, Cork City,  Cork / Dripsey, Cork Cork
524068 2022-12-26 00:00:00 Declan David  Harnett late of Aberdeen, Kitwe, Zambia and,  Dublin 
524069 2023-01-25 00:00:00 Johnny  Mulligan Clooneen, Granard,  Longford Longford
524070 2023-01-25 00:00:00 Mary  McCarthy (née Delahunty) Kilfinane,  Limerick Limerick
524071 2023-01-26 00:00:00 Dessie  Maguire Veldonstown, Brownstown, Navan,  Meath Meath
524072 2023-01-25 00:0

In [7]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

/tmp/ipykernel_680130/3295222312.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()


In [8]:
res

,id,name,date,county,address,notice
0,524061,Lynette Morris (née Betts),2023-01-25,Mayo,"Chestnut Grove, Castlebar, Mayo","Lynette Morris (neé Betts), Chestnut Grove, Ca..."
1,524062,Madeline Bevans,2023-01-26,Dublin,"Drumcondra, Dublin","Bevans Madeline (Drumcondra, Dublin 9) January..."
2,524063,Charlie Keaney,2023-01-25,Cavan,"Derrin, Ballinagh, Cavan","Charlie Keaney, Derrin, Ballinagh, Co. Cavan. ..."
3,524064,Maureen Healy (née Keady),2023-01-25,Galway,"Roscam, Galway / Headford, Galway","Maureen Healy (nee Keady), Roscam, Ballylougha..."
4,524065,Brian Douglas,2023-01-24,Dublin,"Firhouse, Dublin / Booterstown, Dublin","DOUGLAS Brian Richard (Firhouse, D24 and forme..."
...,...,...,...,...,...,...
550,524611,VERONICA O'NEILL (née Motyer),2023-01-29,,"Midleton, Cork",VERONICA O’NEILL (née Motyer) died unexpectedl...
551,524612,Anna Wiseman,2023-01-28,Cork,"Castletownbere, Cork / Terenure, Dublin","WISEMAN, Terenure, Dublin and Castletownbere, ..."
552,524613,Josephine (Josie) O'Shea (née Kelleher),2023-01-30,Cork,"Carrigaphooca, Macroom, Cork","O’Shea (Carrigaphooca, Macroom) on 30th Janua..."
553,524614,LINDA CARSON (née DUFFY),2023-01-29,,"Dromore, Down / Raphoe, Donegal","CARSON, January 29th, 2023, peacefully at home..."


In [9]:
new = pd.concat([df,res]).reset_index(drop=True)
new=new[~new.id.duplicated(keep='first')]
print (len(df),len(res),len(new))
new.to_pickle('rip_dn_scrape.pkl')

523066 555 523621


In [12]:
#x.to_csv('rip_dn_scrape.csv')

## clean data

In [10]:
x=new
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
x=x.drop_duplicates(['name','date','county'])
x = x[~x.address.isnull()]
nc = ['Fermanagh','Armagh','Tyrone','Down','Antrim','Derry']
x = x[~x.county.isin(nc)]
x = x[~x.address.str.contains('|'.join(nc))]
x=x.sort_values('id')
print (len(x))
#x.to_csv('rip_dn_scrape_processed.csv')
x.to_pickle('rip_dn_scrape_processed.pkl')

523621
487204
